# To do

1. Modify the find_num_years_disease function for 1 year 5 year incidence predictions
2. CVD self-reported must be removed from baseline and patients with 1 in any of the corresponding columns must be excluded from the analysis

# Ideas

1. instead of presence or absence of disease use the number of years a disease has existed using the functionfind_num_years_disease (newdf,FID,FID_corr,patientID)
2. Also investigate stroke outcomes, broaden the investigation of outcomes
3. look into crime, mental stress
4. stress,crime, mendelian randomisation
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3989031/
5. 

# Functions
1. find_num_years_disease(newdf,FID,FID_corr,patientID)    
for index specified by patientID, Field ID specified by FID, field ID corresponding to the time event occured and a given patientID specifying the row, it will calculate the difference between event time and age at recruitment from the field ID 21022-0.0
2. tidy_coded_fieldID(bs_html_file,coding,prefix,FID,df)
if a datafield is coded, it will take the beautiful soup object pertaining to the html file (bs_html_file) and the coding of the field ID, e.g. "Coding 1001", and create a tidy table with the column names set to prefix+FID value. df is the df where the FID exists.

# Dictionaries
1. categories: a list of dictionaries in translating categories to fieldIDs in each category
2. fieldID_to_ctgry_dict: reverse of above
3. UID_to_dscrpt: field ID to description and other useful information
4. disease_code_to_disease_dict: disease codes Data coding 6 to disease name for self reported diseases

In [1]:
from IPython.display import display, HTML
import pandas as pd
from lxml import etree
import time
import re
import matplotlib.pyplot as plt
import random
import codecs
import dask.dataframe as dd
import dask.array as da
import numpy as np
import pickle





from bs4 import BeautifulSoup
import requests
from lxml import etree
import pandas as pd
import dask.dataframe as dd
import unicodedata

In [2]:
#PATH_data=r"/home/workspace/datasets/ukbiobank/ukbiobank_yajiez/dataset/ukb26438.csv"
#PATH_dict=r"/home/workspace/datasets/ukbiobank/ukbiobank_yajiez/dataset/ukb26438.html"

PATH_data=r"/home/workspace/datasets/ukbiobank/biobank_milad_34943/ukb34943.csv"
PATH_dict=r"/home/mo/ukb34943.html"

# Read the field ID summary html file

In [3]:
def fielID_to_ctgry(page):
    "The function produces a dictionary from for a given URL above mapping field\
    IDs column[0] to an array with the first element being the description of the\
    field ID and the second element being its corresponding category "
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find(lambda tag: tag.name=='table') 
    rows = table.findAll(lambda tag: tag.name=='tr')
    
    fieldID_to_ctgry_dict=dict()
    for tr in rows:
        td = tr.find_all('td')
        if len(td)>3:
            td.pop()
        row = [tr.text for tr in td]
        try:
            fieldID_to_ctgry_dict[row[0]]=[row[1],unicodedata.normalize("NFKD", row[2]).rstrip()]
        except:
            pass
    return fieldID_to_ctgry_dict


def summarise(PATH_dict):
    """
    takes the dictionary file and produces 3 outputs
    A.UDI_to_dscrpt: a dictionary mapping each UDI to #1: column 
                                                            2: count 
                                                            3: type   
                                                            4: description 
                                                            5: num_instances 
                                                            6: num_Measurements       

    """
    
    html_file = open(PATH_dict)
    html_content = html_file.read()
    parsed_html = etree.HTML(html_content)
    html_tables = parsed_html.findall("body/table")
    
    #This is the main table that contains the field ID descriptions
    pd_code_to_desc=pd.read_html(etree.tostring(html_tables[1]))[0]
    
    #Extract all unique field IDs
    
    tic = time.perf_counter()
    UDI_to_dscrpt=dict()

    for i in range(1,pd_code_to_desc.shape[0]):
        print(pd_code_to_desc.iloc[i][1])
        code_name,inst_num=pd_code_to_desc.iloc[i][1].split('-')
        try:
            UDI_to_dscrpt[code_name]
            inst,num=inst_num.split('.')
            UDI_to_dscrpt[code_name][4:6]=[int(inst), int(num)]
        except:
            UDI_to_dscrpt[code_name]=[pd_code_to_desc.iloc[i][0],
                                                pd_code_to_desc.iloc[i][2],
                                                   pd_code_to_desc.iloc[i][3],
                                                   pd_code_to_desc.iloc[i][4],
                                                   0,
                                                   0]#1: column 2: count 3: type   4: description 5: instances 6: num_Measurements       

    toc = time.perf_counter()   
    print('Time elapsed for extracting unique field IDs: '+str(toc-tic))
    print('The number of unique field IDs: '+str(len(UDI_to_dscrpt)))
  
    return UDI_to_dscrpt


def get_categories(UID_to_dscrpt,fieldID_to_ctgry_dict):
        """The produces a dictionary with the categories present in the dataset as the keys
        and values being an array of field IDs for each category in the dataset as peresent in UID_to_dscrpt
        field_ID_no_category shows the list of field IDs that were present in the dataset (UID_to_dscrpt) 
        but were not found in the URLS and therefore have not categories"""
        categories=dict()
        fieldID_no_category=dict() #a dictionary containing field IDs that do not appear in the URLs and therefore have no categories
        for fieldID in UID_to_dscrpt.keys():
            try:
                categories[fieldID_to_ctgry_dict[fieldID][1]]=[]
            except:
                print('Field ID '+str(fieldID)+' did not exist in the dictionary!')
                print(' ** \t '+UID_to_dscrpt[fieldID][3]+' ')
                fieldID_no_category[fieldID]=UID_to_dscrpt[fieldID][3]

        for fieldID in UID_to_dscrpt.keys():
            try:
                categories[fieldID_to_ctgry_dict[fieldID][1]].append(fieldID+': '+UID_to_dscrpt[fieldID][3])
            except:
                pass
        return categories,fieldID_no_category

def findWholeWord(w):
    #\b({0})\b'
    return re.compile(r'({0})'.format(w), flags=re.IGNORECASE).search 


def search_description(word,UDI_to_dscrpt):
    """
    This function searches the description element of the UDI_to_dscrpt dictionary for a word
    """
    for key in UDI_to_dscrpt.keys():
        if findWholeWord(word)(UDI_to_dscrpt[key][3]):
            print('Field ID: '+key+' _ '+UDI_to_dscrpt[key][3]+'\n')

In [4]:
URLS=['http://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=0&vt=11',
     'http://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=0&vt=21',
     'http://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=0&vt=22' ,
      'http://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=0&vt=31' ,
      'http://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=0&vt=41' ,
      'http://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=0&vt=51' ,
      'http://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=0&vt=61' ,
      'http://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=0&vt=101',
      'https://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=20&vt=-1',
      'https://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=10&vt=-1',
      'https://biobank.ctsu.ox.ac.uk/crystal/list.cgi?it=30&vt=-1']

In [5]:
fieldID_to_ctgry_dict=dict()
#compile the dictionary for mapping fieldIDs to categories form the URLs
for url in URLS:
    page = requests.get(url)
    fieldID_to_ctgry_dict={**fieldID_to_ctgry_dict,**fielID_to_ctgry(page)}
print('fieldID_to_ctgry_dict compiled from URLS with '+str(len(fieldID_to_ctgry_dict.keys()))+'field IDs')

#get a list unique field IDs in the html file with their descriptions and other descriptive stats
UID_to_dscrpt=summarise(PATH_dict)   

#map filed IDs in the data to categories and show as dict
categories,fieldID_no_category=get_categories(UID_to_dscrpt,fieldID_to_ctgry_dict)

word='diagnoses'
search_description(word,UID_to_dscrpt)

fieldID_to_ctgry_dict compiled from URLS with 7806field IDs
3-0.0
3-1.0
3-2.0
4-0.0
4-1.0
4-2.0
5-0.0
5-1.0
5-2.0
6-0.0
6-1.0
6-2.0
21-0.0
21-1.0
21-2.0
31-0.0
34-0.0
36-0.0
36-1.0
36-2.0
37-0.0
37-1.0
37-2.0
38-0.0
38-1.0
38-2.0
39-0.0
39-1.0
39-2.0
40-0.0
40-1.0
40-2.0
41-0.0
41-1.0
41-2.0
43-0.0
43-1.0
43-2.0
44-0.0
44-1.0
44-2.0
46-0.0
46-1.0
46-2.0
47-0.0
47-1.0
47-2.0
48-0.0
48-1.0
48-2.0
49-0.0
49-1.0
49-2.0
50-0.0
50-1.0
50-2.0
51-0.0
51-1.0
51-2.0
52-0.0
53-0.0
53-1.0
53-2.0
54-0.0
54-1.0
54-2.0
55-0.0
55-1.0
55-2.0
84-0.0
84-0.1
84-0.2
84-0.3
84-0.4
84-0.5
84-1.0
84-1.1
84-1.2
84-1.3
84-1.4
84-1.5
84-2.0
84-2.1
84-2.2
84-2.3
84-2.4
84-2.5
87-0.0
87-0.1
87-0.2
87-0.3
87-0.4
87-0.5
87-0.6
87-0.7
87-0.8
87-0.9
87-0.10
87-0.11
87-0.12
87-0.13
87-0.14
87-0.15
87-0.16
87-0.17
87-0.18
87-0.19
87-0.20
87-0.21
87-0.22
87-0.23
87-0.24
87-0.25
87-0.26
87-0.27
87-0.28
87-0.29
87-0.30
87-0.31
87-0.32
87-0.33
87-1.0
87-1.1
87-1.2
87-1.3
87-1.4
87-1.5
87-1.6
87-1.7
87-1.8
87-1.9
87-1.10
87-

In [6]:
UID_to_dscrpt

{'3': [1, 568178, 'Integer, seconds', 'Verbal interview duration', 2, 0],
 '4': [4, 568066, 'Integer, seconds', 'Biometrics duration', 2, 0],
 '5': [7, 573040, 'Integer, seconds', 'Sample collection duration', 2, 0],
 '6': [10, 570625, 'Integer, seconds', 'Conclusion duration', 2, 0],
 '21': [13,
  571813,
  'Categorical (single)',
  'Weight methodUses data-coding 100261',
  2,
  0],
 '31': [16, 502489, 'Categorical (single)', 'SexUses data-coding 9', 0, 0],
 '34': [17, 502489, 'Integer, years', 'Year of birth', 0, 0],
 '36': [18, 564409, 'Text', 'Blood pressure device ID', 2, 0],
 '37': [21,
  564409,
  'Text',
  'Blood pressure manual sphygmomanometer device ID',
  2,
  0],
 '38': [24, 568067, 'Text', 'Hand grip dynamometer device ID', 2, 0],
 '39': [27, 568067, 'Text', 'Height measure device ID', 2, 0],
 '40': [30, 568067, 'Text', 'Manual scales device ID', 2, 0],
 '41': [33, 568067, 'Text', 'Seating box device ID', 2, 0],
 '43': [36, 568067, 'Text', 'Impedance device ID', 2, 0],
 '

In [7]:
for category in categories.keys():
    print('Category: '+category)
    for fieldID in categories[category]:
        print('\t \t '+fieldID)

Category: Body size measures
	 	 21: Weight methodUses data-coding 100261
	 	 48: Waist circumference
	 	 49: Hip circumference
	 	 50: Standing height
	 	 51: Seated height
	 	 3077: Seating box heightUses data-coding 100262
	 	 3160: Weight, manual entry
	 	 12143: Weight (pre-imaging)
	 	 12144: Height
	 	 20015: Sitting height
	 	 20041: Reason for skipping weightUses data-coding 47
	 	 20045: Reason for skipping waistUses data-coding 47
	 	 20046: Reason for skipping hip measurementUses data-coding 47
	 	 20047: Reason for skipping standing heightUses data-coding 47
	 	 20048: Reason for skipping sitting heightUses data-coding 47
	 	 21001: Body mass index (BMI)
	 	 21002: Weight
Category: Baseline characteristics
	 	 31: SexUses data-coding 9
	 	 34: Year of birth
	 	 52: Month of birthUses data-coding 8
	 	 189: Townsend deprivation index at recruitment
	 	 21022: Age at recruitment
Category: Hand grip strength
	 	 46: Hand grip strength (left)
	 	 47: Hand grip strength (right)

In [8]:
categories['Baseline characteristics']

['31: SexUses data-coding 9',
 '34: Year of birth',
 '52: Month of birthUses data-coding 8',
 '189: Townsend deprivation index at recruitment',
 '21022: Age at recruitment']

# 2. Read data as pd for correctness check

## 2.1 read datafields by categories in numpy

In [ ]:
tic = time.perf_counter()
column_names = pd.read_csv(PATH_data, dtype='unicode',nrows=1).columns
df = pd.read_csv(PATH_data,dtype='unicode',engine='python')
#df=next(df)
toc = time.perf_counter()
print('Time to read the data: '+str(toc-tic))
df

# 2.2 Read Demographics (baseline characteristics) and socioeconomics and  store 

## 2.2.1 Read Socio Demographics

In [17]:
selected_categories={'Baseline characteristics':['include','31','34','52','189','21022']}
process_config={'31':{'type':'int'},'34':{'type':'string'},'52':{'type':'string'},'189':{'type':'float'},'21022':{'type':'int'}}

def preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config):
    class CustomError(Exception):
        pass
    new_UID_to_dscrpt=dict()
    newdf=pd.DataFrame()
    #run through each category go through selected_categories and determine what field IDs to select
    newdf_col_cnt=0
    for cat in selected_categories.keys():
        selection_type=selected_categories[cat][0]
        listed_fieldIDs=set(selected_categories[cat][1:])
        accepted_values=('include','exclude','include all','exclude all')
        if selection_type not in accepted_values:
            raise ValueError('The first element of the selected_categories dictionary array for the category "{}" needs\
            to be one of the values below \n{}.\nThe input value is "{}".'.format(cat,accepted_values,selection_type))    
        
        #run through each category go through selected_categories and determine what field IDs to select
        all_fieldIDs={fieldID.split(':')[0] for fieldID in categories[cat]}
        if selection_type=='include all':
            selected_fieldIDs=all_fieldIDs
        elif selection_type=='exclude all':
            selected_fieldIDs=[]
        elif selection_type=='include':
            if listed_fieldIDs.issubset(all_fieldIDs):
                selected_fieldIDs=listed_fieldIDs
            else:
                raise ValueError('The field IDs selected to be included are not valid.')
        elif selection_type=='exclude':
            if listed_fieldIDs.issubset(all_fieldIDs):
                selected_fieldIDs=all_fieldIDs.difference(listed_fieldIDs)
            else: 
                raise ValueError('The field IDs selected to be excluded are not valid.')
        else:
            raise CustomError('This has happened due to a bug, investigate!')

        print('From the category {} the following where selected: \n \t {}'.format(cat,selected_fieldIDs))
        for FID in selected_fieldIDs:
            print('\t FID {}:  {}'.format(FID,UID_to_dscrpt[FID][3]))

        #
        for FID in selected_fieldIDs:
            newdf=pd.concat([newdf,df.iloc[:,UID_to_dscrpt[FID][0]:UID_to_dscrpt[FID][0]+(UID_to_dscrpt[FID][4]+1)*(UID_to_dscrpt[FID][5]+1)].astype(process_config[FID]['type'])],axis=1)
            newdf_col_cnt+=1
    #display(HTML(newdf.head().to_html()))
    return newdf


newdf=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

"""#Next grab 20002 (self-reported non-cancer) and 20009 (age of self-reported non-cancer) 
and seperate illnesses based on the time of occurance to and time window"""

newdf['birth_date']=pd.to_datetime(newdf['34-0.0']+newdf['52-0.0'],format='%Y%m')
newdf.drop(columns=['34-0.0','52-0.0'],inplace=True)
newdf


From the category Baseline characteristics the following where selected: 
 	 {'189', '31', '52', '21022', '34'}
	 FID 189:  Townsend deprivation index at recruitment
	 FID 31:  SexUses data-coding 9
	 FID 52:  Month of birthUses data-coding 8
	 FID 21022:  Age at recruitment
	 FID 34:  Year of birth


,189-0.0,31-0.0,21022-0.0,birth_date
0,-0.953776,0,62,1947-02-01
1,5.116970,0,56,1952-12-01
2,-2.322980,0,49,1960-10-01
3,-0.804055,1,47,1962-05-01
4,-0.346170,1,67,1941-02-01
...,...,...,...,...
9995,-1.855400,0,41,1968-09-01
9996,4.521070,1,63,1946-09-01
9997,-0.139468,1,52,1955-04-01
9998,-1.152500,1,62,1946-02-01


In [18]:
selected_categories_tidy={'demographics and socioeconomics':['gender',
                                                      'birth_date',
                                                      'Townsend deprivation index at recruitment',
                                                      'Age at recruitment']}
selected_categories_tidy_columns={'gender':{"col_name":['31-0.0'],"type":"binary"},
                                  'birth_date':{"col_name":['birth_date'],"type":"date"},
                                  'Townsend deprivation index at recruitment':{"col_name":['189-0.0'],"type":"float"},
                                  'Age at recruitment':{"col_name":['21022-0.0'],"type":"int"}}
selected_categories_exclusions=[]

In [19]:
word='doctor'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['6150'])
categories['Medical conditions']

Field ID: 2090 _ Seen doctor (GP) for nerves, anxiety, tension or depressionUses data-coding 100484

Field ID: 2443 _ Diabetes diagnosed by doctorUses data-coding 100603

Field ID: 2453 _ Cancer diagnosed by doctorUses data-coding 100603

Field ID: 2473 _ Other serious medical condition/disability diagnosed by doctorUses data-coding 100603

Field ID: 6150 _ Vascular/heart problems diagnosed by doctorUses data-coding 100605

Field ID: 6152 _ Blood clot, DVT, bronchitis, emphysema, asthma, rhinitis, eczema, allergy diagnosed by doctorUses data-coding 100610

Field ID: 20491 _ Someone to take to doctor when needed as a childUses data-coding 532

Field ID: 22126 _ Doctor diagnosed hayfever or allergic rhinitisUses data-coding 100700

Field ID: 22127 _ Doctor diagnosed asthmaUses data-coding 100700

Field ID: 22128 _ Doctor diagnosed emphysemaUses data-coding 100700

Field ID: 22129 _ Doctor diagnosed chronic bronchitisUses data-coding 100700

Field ID: 22130 _ Doctor diagnosed COPD (chroni

['84: Cancer year/age first occurredUses data-coding 37',
 '87: Non-cancer illness year/age first occurredUses data-coding 37',
 '134: Number of self-reported cancers',
 '135: Number of self-reported non-cancer illnesses',
 '2443: Diabetes diagnosed by doctorUses data-coding 100603',
 '2453: Cancer diagnosed by doctorUses data-coding 100603',
 '2463: Fractured/broken bones in last 5 yearsUses data-coding 100603',
 '2473: Other serious medical condition/disability diagnosed by doctorUses data-coding 100603',
 '2966: Age high blood pressure diagnosedUses data-coding 100369',
 '2976: Age diabetes diagnosedUses data-coding 100369',
 '2986: Started insulin within one year diagnosis of diabetesUses data-coding 100369',
 '3005: Fracture resulting from simple fallUses data-coding 100369',
 '3140: PregnantUses data-coding 100267',
 '3627: Age angina diagnosedUses data-coding 100563',
 '3761: Age hay fever, rhinitis or eczema diagnosedUses data-coding 100564',
 '3786: Age asthma diagnosedUses da

## 2.2.2 Read Ethinicity and add 

In [20]:
word='ethnic'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['21000'])
categories['Ethnicity']

Field ID: 21000 _ Ethnic backgroundUses data-coding 1001

Field ID: 22006 _ Genetic ethnic groupingUses data-coding 1002

None
['Ethnic background', 'Ethnicity']


['3659: Year immigrated to UK (United Kingdom)Uses data-coding 100563',
 '21000: Ethnic backgroundUses data-coding 1001']

In [21]:
selected_categories={'Ethnicity':['include','21000']}
process_config={'21000':{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

From the category Ethnicity the following where selected: 
 	 {'21000'}
	 FID 21000:  Ethnic backgroundUses data-coding 1001


In [22]:
df_temp

,21000-0.0,21000-1.0,21000-2.0
0,1001,<NA>,<NA>
1,1001,1001,1001
2,1001,<NA>,<NA>
3,1001,<NA>,<NA>
4,1001,<NA>,<NA>
...,...,...,...
9995,1001,<NA>,<NA>
9996,1002,<NA>,<NA>
9997,1001,<NA>,<NA>
9998,1001,<NA>,<NA>


In [23]:
df_temp=df_temp[['21000-0.0']]

In [24]:
def tidy_coded_fieldID(bs_html_file,coding,prefix,FID,df):
    table = bs_html_file.find(lambda tag: tag.name=='table' and tag.has_attr('summary') and tag['summary']==coding)
    
    pd_coding=pd.read_html(str(table))[0]
    
    coded_entry_to_code_dict=pd_coding[['Code','Meaning']].set_index('Code').to_dict()['Meaning']
    coded_entry_to_code_dict={str(k):prefix+v for k,v in coded_entry_to_code_dict.items()}
    
    col_names=list(coded_entry_to_code_dict.values())#+disease_code_to_disease_dict_time
    col_names.sort()
    df_tidy=pd.DataFrame(index=newdf.index,columns=col_names)
    
    #grab columns corresponding to the first assessment x-0.y
    
    for patientID in df.index:
        cols_FID=[col for col in df.columns if (FID in col) and col.split('-')[1].split('.')[0]=='0']
        for col in cols_FID:
            entry=df.loc[patientID,col]
            if not pd.isna(entry):
                df_tidy.loc[patientID,coded_entry_to_code_dict[entry]]=1
    df_tidy.fillna(0,inplace=True)
    print(df_tidy.sum())

    
    return df_tidy
bs = BeautifulSoup(open(PATH_dict).read(), 'html.parser')
df_tidy=tidy_coded_fieldID(bs,"Coding 1001","Ethnicity_","21000",df_temp)

Ethnicity_African                         72
Ethnicity_Any other Asian background      41
Ethnicity_Any other Black background       8
Ethnicity_Any other mixed background      20
Ethnicity_Any other white background     324
Ethnicity_Asian or Asian British           2
Ethnicity_Bangladeshi                      4
Ethnicity_Black or Black British           0
Ethnicity_British                       8779
Ethnicity_Caribbean                       95
Ethnicity_Chinese                         28
Ethnicity_Do not know                      4
Ethnicity_Indian                         130
Ethnicity_Irish                          245
Ethnicity_Mixed                            1
Ethnicity_Other ethnic group              95
Ethnicity_Pakistani                       39
Ethnicity_Prefer not to answer            31
Ethnicity_White                           13
Ethnicity_White and Asian                 22
Ethnicity_White and Black African          7
Ethnicity_White and Black Caribbean       19
dtype: int

In [25]:
newdf=pd.concat([newdf,df_tidy],axis=1)

In [26]:
newdf

,189-0.0,31-0.0,21022-0.0,birth_date,Ethnicity_African,Ethnicity_Any other Asian background,Ethnicity_Any other Black background,Ethnicity_Any other mixed background,Ethnicity_Any other white background,Ethnicity_Asian or Asian British,...,Ethnicity_Indian,Ethnicity_Irish,Ethnicity_Mixed,Ethnicity_Other ethnic group,Ethnicity_Pakistani,Ethnicity_Prefer not to answer,Ethnicity_White,Ethnicity_White and Asian,Ethnicity_White and Black African,Ethnicity_White and Black Caribbean
0,-0.953776,0,62,1947-02-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5.116970,0,56,1952-12-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-2.322980,0,49,1960-10-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.804055,1,47,1962-05-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-0.346170,1,67,1941-02-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-1.855400,0,41,1968-09-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,4.521070,1,63,1946-09-01,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9997,-0.139468,1,52,1955-04-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,-1.152500,1,62,1946-02-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
selected_categories_tidy['demographics and socioeconomics'].append("Ethnicity")
selected_categories_tidy_columns['Ethnicity']={"col_name":list(df_tidy.columns.values),"type":"binary"}

## 2.2.3 Read and add socioeconomics
1. household income before tax, 
2. current employment status,
3. Townsend deprivation index (done in baseline) at recruitment, 
4. education qualifications 

In [28]:
word='household'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['738'])
categories['Household']

Field ID: 709 _ Number in householdUses data-coding 100291

Field ID: 728 _ Number of vehicles in householdUses data-coding 100293

Field ID: 738 _ Average total household income before taxUses data-coding 100294

Field ID: 1259 _ Smoking/smokers in householdUses data-coding 100370

Field ID: 6141 _ How are people in household related to participantUses data-coding 100292

Field ID: 10877 _ Average total household income before tax (pilot)Uses data-coding 100657

None
['Average total household income before tax', 'Household']


['670: Type of accommodation lived inUses data-coding 100286',
 '680: Own or rent accommodation lived inUses data-coding 100287',
 '699: Length of time at current addressUses data-coding 100290',
 '709: Number in householdUses data-coding 100291',
 '728: Number of vehicles in householdUses data-coding 100293',
 '738: Average total household income before taxUses data-coding 100294',
 '6139: Gas or solid-fuel cooking/heatingUses data-coding 100288',
 '6140: Heating type(s) in homeUses data-coding 100289',
 '6141: How are people in household related to participantUses data-coding 100292',
 '10860: Gas or solid-fuel cooking/heating (pilot)Uses data-coding 100656',
 '10877: Average total household income before tax (pilot)Uses data-coding 100657']

In [29]:
selected_categories={'Household':['include','738']}
process_config={'738':{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)
print(df_temp)
df_temp=df_temp[['738-0.0']]

df_tidy=tidy_coded_fieldID(bs,"Coding 100294","Household_inc_","738",df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy['demographics and socioeconomics'].append("Household Income")
selected_categories_tidy_columns['Household Income']={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Household the following where selected: 
 	 {'738'}
	 FID 738:  Average total household income before taxUses data-coding 100294
     738-0.0 738-1.0 738-2.0
0          3    <NA>    <NA>
1          3       2       3
2         -1    <NA>      -1
3          4    <NA>    <NA>
4         -3    <NA>    <NA>
...      ...     ...     ...
9995       4    <NA>    <NA>
9996       1    <NA>    <NA>
9997       3    <NA>    <NA>
9998       1    <NA>    <NA>
9999       1    <NA>    <NA>

[10000 rows x 3 columns]
Household_inc_18,000 to 30,999        2192
Household_inc_31,000 to 51,999        2203
Household_inc_52,000 to 100,000       1766
Household_inc_Do not know              386
Household_inc_Greater than 100,000     437
Household_inc_Less than 18,000        1928
Household_inc_Prefer not to answer     964
dtype: int64


In [30]:
word='employment'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['20119'])
categories['Employment']

Field ID: 6142 _ Current employment statusUses data-coding 100295

Field ID: 20119 _ Current employment status - correctedUses data-coding 91

Field ID: 26412 _ Employment score (England)

Field ID: 26419 _ Employment score (Wales)

None
['Current employment status - corrected', 'Employment']


['132: Job code at visit - enteredUses data-coding 170',
 '757: Time employed in main current jobUses data-coding 100294',
 '767: Length of working week for main jobUses data-coding 100294',
 '777: Frequency of travelling from home to job workplaceUses data-coding 100298',
 '796: Distance between home and job workplaceUses data-coding 100300',
 '806: Job involves mainly walking or standingUses data-coding 100301',
 '816: Job involves heavy manual or physical workUses data-coding 100301',
 '826: Job involves shift workUses data-coding 100301',
 '3426: Job involves night shift workUses data-coding 100267',
 '6142: Current employment statusUses data-coding 100295',
 '6143: Transport type for commuting to job workplaceUses data-coding 100299',
 '20119: Current employment status - correctedUses data-coding 91',
 '20121: Cascot confidence score']

In [31]:
FID='6142'
data_coding='100295'
FID_tidy_name="Current employment status"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,"Employment_",FID,df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy['demographics and socioeconomics'].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Employment the following where selected: 
 	 {'6142'}
	 FID 6142:  Current employment statusUses data-coding 100295
     6142-0.0 6142-0.1 6142-0.2 6142-0.3 6142-0.4 6142-0.5 6142-0.6 6142-1.0  \
0           2     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
1           1     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>        2   
2           3     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
3           1     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
4           2     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
...       ...      ...      ...      ...      ...      ...      ...      ...   
9995        1     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9996        4     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9997        1     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9998        3     <NA>     <NA>     <NA>     <NA>     <NA>     <NA

In [32]:
word='qualification'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['6138'])
categories['Education']

Field ID: 6138 _ QualificationsUses data-coding 100305

Field ID: 10722 _ Qualifications (pilot)Uses data-coding 100658

None
['Qualifications', 'Education']


['845: Age completed full time educationUses data-coding 100306',
 '6138: QualificationsUses data-coding 100305',
 '10722: Qualifications (pilot)Uses data-coding 100658']

In [33]:
FID='6138'
data_coding='100305'
FID_tidy_name="Qualifications"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,"Qualifications_",FID,df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy['demographics and socioeconomics'].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Education the following where selected: 
 	 {'6138'}
	 FID 6138:  QualificationsUses data-coding 100305
     6138-0.0 6138-0.1 6138-0.2 6138-0.3 6138-0.4 6138-0.5 6138-1.0 6138-1.1  \
0           1        6     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
1           1        2        3        6     <NA>     <NA>        1        2   
2           1        3        6     <NA>     <NA>     <NA>     <NA>     <NA>   
3           1     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
4          -7     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
...       ...      ...      ...      ...      ...      ...      ...      ...   
9995        3        4     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9996        3     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9997        2        3     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9998       -7     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>  

In [34]:
newdf.to_csv(r'tidy_data_tiles/sociodem_socioeco.csv')

In [35]:
newdf

,189-0.0,31-0.0,21022-0.0,birth_date,Ethnicity_African,Ethnicity_Any other Asian background,Ethnicity_Any other Black background,Ethnicity_Any other mixed background,Ethnicity_Any other white background,Ethnicity_Asian or Asian British,...,Employment_Unable to work because of sickness or disability,Employment_Unemployed,Qualifications_A levels/AS levels or equivalent,Qualifications_CSEs or equivalent,Qualifications_College or University degree,Qualifications_NVQ or HND or HNC or equivalent,Qualifications_None of the above,Qualifications_O levels/GCSEs or equivalent,"Qualifications_Other professional qualifications eg: nursing, teaching",Qualifications_Prefer not to answer
0,-0.953776,0,62,1947-02-01,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,5.116970,0,56,1952-12-01,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,-2.322980,0,49,1960-10-01,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,-0.804055,1,47,1962-05-01,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,-0.346170,1,67,1941-02-01,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-1.855400,0,41,1968-09-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9996,4.521070,1,63,1946-09-01,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
9997,-0.139468,1,52,1955-04-01,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9998,-1.152500,1,62,1946-02-01,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [36]:
selected_categories_tidy_columns

{'gender': {'col_name': ['31-0.0'], 'type': 'binary'},
 'birth_date': {'col_name': ['birth_date'], 'type': 'date'},
 'Townsend deprivation index at recruitment': {'col_name': ['189-0.0'],
  'type': 'float'},
 'Age at recruitment': {'col_name': ['21022-0.0'], 'type': 'int'},
 'Ethnicity': {'col_name': ['Ethnicity_African',
   'Ethnicity_Any other Asian background',
   'Ethnicity_Any other Black background',
   'Ethnicity_Any other mixed background',
   'Ethnicity_Any other white background',
   'Ethnicity_Asian or Asian British',
   'Ethnicity_Bangladeshi',
   'Ethnicity_Black or Black British',
   'Ethnicity_British',
   'Ethnicity_Caribbean',
   'Ethnicity_Chinese',
   'Ethnicity_Do not know',
   'Ethnicity_Indian',
   'Ethnicity_Irish',
   'Ethnicity_Mixed',
   'Ethnicity_Other ethnic group',
   'Ethnicity_Pakistani',
   'Ethnicity_Prefer not to answer',
   'Ethnicity_White',
   'Ethnicity_White and Asian',
   'Ethnicity_White and Black African',
   'Ethnicity_White and Black Caribbe

# 2.3 Anthropometrics and cardiac risk factors
1. Standing Height, 
2. BMI,
3. SBP, DBP, PR 
4. Drinking, and Smoking status

In [37]:
newdf=pd.DataFrame()

In [38]:
word='standing height'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['50'])
categories['Body size measures']

Field ID: 50 _ Standing height

Field ID: 20047 _ Reason for skipping standing heightUses data-coding 47

None
['Standing height', 'Body size measures']


['21: Weight methodUses data-coding 100261',
 '48: Waist circumference',
 '49: Hip circumference',
 '50: Standing height',
 '51: Seated height',
 '3077: Seating box heightUses data-coding 100262',
 '3160: Weight, manual entry',
 '12143: Weight (pre-imaging)',
 '12144: Height',
 '20015: Sitting height',
 '20041: Reason for skipping weightUses data-coding 47',
 '20045: Reason for skipping waistUses data-coding 47',
 '20046: Reason for skipping hip measurementUses data-coding 47',
 '20047: Reason for skipping standing heightUses data-coding 47',
 '20048: Reason for skipping sitting heightUses data-coding 47',
 '21001: Body mass index (BMI)',
 '21002: Weight']

In [39]:
FID='50'
#data_coding='100261'
FID_tidy_name="Standing height"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"int"}

From the category Body size measures the following where selected: 
 	 {'50'}
	 FID 50:  Standing height
     50-0.0 50-1.0 50-2.0
0       154   <NA>   <NA>
1       174    172    173
2       166   <NA>    166
3       179   <NA>   <NA>
4       168   <NA>   <NA>
...     ...    ...    ...
9995    161   <NA>   <NA>
9996    176   <NA>   <NA>
9997    181   <NA>   <NA>
9998    181   <NA>   <NA>
9999    156   <NA>   <NA>

[10000 rows x 3 columns]


In [40]:
word='BMI'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['21001'])
categories['Body size measures']

Field ID: 21001 _ Body mass index (BMI)

Field ID: 23104 _ Body mass index (BMI)

None
['Body mass index (BMI)', 'Body size measures']


['21: Weight methodUses data-coding 100261',
 '48: Waist circumference',
 '49: Hip circumference',
 '50: Standing height',
 '51: Seated height',
 '3077: Seating box heightUses data-coding 100262',
 '3160: Weight, manual entry',
 '12143: Weight (pre-imaging)',
 '12144: Height',
 '20015: Sitting height',
 '20041: Reason for skipping weightUses data-coding 47',
 '20045: Reason for skipping waistUses data-coding 47',
 '20046: Reason for skipping hip measurementUses data-coding 47',
 '20047: Reason for skipping standing heightUses data-coding 47',
 '20048: Reason for skipping sitting heightUses data-coding 47',
 '21001: Body mass index (BMI)',
 '21002: Weight']

In [41]:
FID='21001'
#data_coding='100261'
FID_tidy_name="BMI"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"float"}

From the category Body size measures the following where selected: 
 	 {'21001'}
	 FID 21001:  Body mass index (BMI)
     21001-0.0 21001-1.0 21001-2.0
0      29.9798      <NA>      <NA>
1      24.4088   23.5938   22.5868
2      29.3584      <NA>   37.5236
3       30.617      <NA>      <NA>
4      31.3209      <NA>      <NA>
...        ...       ...       ...
9995   21.8356      <NA>      <NA>
9996   33.6067      <NA>      <NA>
9997   28.6011      <NA>      <NA>
9998   29.6999      <NA>      <NA>
9999   23.1345      <NA>      <NA>

[10000 rows x 3 columns]


In [42]:
word='systolic'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['4080'])
categories['Blood pressure']

Field ID: 93 _ Systolic blood pressure, manual reading

Field ID: 4080 _ Systolic blood pressure, automated reading

None
['Systolic blood pressure, automated reading', 'Blood pressure']


['93: Systolic blood pressure, manual reading',
 '94: Diastolic blood pressure, manual reading',
 '95: Pulse rate (during blood-pressure measurement)',
 '102: Pulse rate, automated reading',
 '4079: Diastolic blood pressure, automated reading',
 '4080: Systolic blood pressure, automated reading']

In [43]:
FID='4080'
#data_coding='100261'
FID_tidy_name="SBP"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0',FID+'-0.1']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])


df_temp[FID+'-0.0']= pd.to_numeric(df_temp[FID+'-0.0'], errors='coerce')
df_temp[FID+'-0.1']= pd.to_numeric(df_temp[FID+'-0.1'], errors='coerce')

df_temp.fillna(df_temp.mean(),inplace=True)

df_temp['SBP_avg']=df_temp.mean(axis=1)
df_temp.drop(columns=[FID+'-0.0',FID+'-0.1'],inplace=True)

df_temp=df_temp.round(0).astype(int)

df_temp
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"int"}

From the category Blood pressure the following where selected: 
 	 {'4080'}
	 FID 4080:  Systolic blood pressure, automated reading
     4080-0.0 4080-0.1 4080-1.0 4080-1.1 4080-2.0 4080-2.1
0         138      134     <NA>     <NA>     <NA>     <NA>
1         129      119      124      114     <NA>     <NA>
2         125      113     <NA>     <NA>     <NA>     <NA>
3         169      171     <NA>     <NA>     <NA>     <NA>
4        <NA>     <NA>     <NA>     <NA>     <NA>     <NA>
...       ...      ...      ...      ...      ...      ...
9995      116      113     <NA>     <NA>     <NA>     <NA>
9996      167      137     <NA>     <NA>     <NA>     <NA>
9997      151      145     <NA>     <NA>     <NA>     <NA>
9998      164      154     <NA>     <NA>     <NA>     <NA>
9999      141      165     <NA>     <NA>     <NA>     <NA>

[10000 rows x 6 columns]
the percentage of na
4080-0.0    0.0644
4080-0.1    0.0785
dtype: float64


In [44]:
word='diastolic'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['4079'])
categories['Blood pressure']

Field ID: 94 _ Diastolic blood pressure, manual reading

Field ID: 4079 _ Diastolic blood pressure, automated reading

None
['Diastolic blood pressure, automated reading', 'Blood pressure']


['93: Systolic blood pressure, manual reading',
 '94: Diastolic blood pressure, manual reading',
 '95: Pulse rate (during blood-pressure measurement)',
 '102: Pulse rate, automated reading',
 '4079: Diastolic blood pressure, automated reading',
 '4080: Systolic blood pressure, automated reading']

In [45]:
FID='4079'
#data_coding='100261'
FID_tidy_name="DBP"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0',FID+'-0.1']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])


df_temp[FID+'-0.0']= pd.to_numeric(df_temp[FID+'-0.0'], errors='coerce')
df_temp[FID+'-0.1']= pd.to_numeric(df_temp[FID+'-0.1'], errors='coerce')

df_temp.fillna(df_temp.mean(),inplace=True)

df_temp['DBP_avg']=df_temp.mean(axis=1)
df_temp.drop(columns=[FID+'-0.0',FID+'-0.1'],inplace=True)

df_temp=df_temp.round(0).astype(int)

df_temp
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"int"}

From the category Blood pressure the following where selected: 
 	 {'4079'}
	 FID 4079:  Diastolic blood pressure, automated reading
     4079-0.0 4079-0.1 4079-1.0 4079-1.1 4079-2.0 4079-2.1
0          74       76     <NA>     <NA>     <NA>     <NA>
1          76       79       71       71     <NA>     <NA>
2          76       77     <NA>     <NA>     <NA>     <NA>
3         107      107     <NA>     <NA>     <NA>     <NA>
4        <NA>     <NA>     <NA>     <NA>     <NA>     <NA>
...       ...      ...      ...      ...      ...      ...
9995       61       67     <NA>     <NA>     <NA>     <NA>
9996       84       73     <NA>     <NA>     <NA>     <NA>
9997       88       88     <NA>     <NA>     <NA>     <NA>
9998       93       91     <NA>     <NA>     <NA>     <NA>
9999       72       76     <NA>     <NA>     <NA>     <NA>

[10000 rows x 6 columns]
the percentage of na
4079-0.0    0.0643
4079-0.1    0.0785
dtype: float64


In [46]:
FID='102'
#data_coding='100261'
FID_tidy_name="PR"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0',FID+'-0.1']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])


df_temp[FID+'-0.0']= pd.to_numeric(df_temp[FID+'-0.0'], errors='coerce')
df_temp[FID+'-0.1']= pd.to_numeric(df_temp[FID+'-0.1'], errors='coerce')

df_temp.fillna(df_temp.mean(),inplace=True)

df_temp['PR_avg']=df_temp.mean(axis=1)
df_temp.drop(columns=[FID+'-0.0',FID+'-0.1'],inplace=True)

df_temp=df_temp.round(0).astype(int)

df_temp
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"int"}

From the category Blood pressure the following where selected: 
 	 {'102'}
	 FID 102:  Pulse rate, automated reading
     102-0.0 102-0.1 102-1.0 102-1.1 102-2.0 102-2.1
0         83      82    <NA>    <NA>    <NA>    <NA>
1         67      64      53      55    <NA>    <NA>
2         57      60    <NA>    <NA>    <NA>    <NA>
3         57      56    <NA>    <NA>    <NA>    <NA>
4       <NA>    <NA>    <NA>    <NA>    <NA>    <NA>
...      ...     ...     ...     ...     ...     ...
9995      64      68    <NA>    <NA>    <NA>    <NA>
9996      76      76    <NA>    <NA>    <NA>    <NA>
9997      94      95    <NA>    <NA>    <NA>    <NA>
9998      86      86    <NA>    <NA>    <NA>    <NA>
9999      65      64    <NA>    <NA>    <NA>    <NA>

[10000 rows x 6 columns]
the percentage of na
102-0.0    0.0643
102-0.1    0.0785
dtype: float64


In [47]:
newdf

,50-0.0,21001-0.0,SBP_avg,DBP_avg,PR_avg
0,154,29.9798,136,75,82
1,174,24.4088,124,78,66
2,166,29.3584,119,76,58
3,179,30.617,170,107,56
4,168,31.3209,138,82,69
...,...,...,...,...,...
9995,161,21.8356,114,64,66
9996,176,33.6067,152,78,76
9997,181,28.6011,148,88,94
9998,181,29.6999,159,92,86


In [48]:
word='alcohol'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['20117'])
categories['Alcohol']

Field ID: 1558 _ Alcohol intake frequency.Uses data-coding 100402

Field ID: 1618 _ Alcohol usually taken with mealsUses data-coding 100416

Field ID: 1628 _ Alcohol intake versus 10 years previouslyUses data-coding 100417

Field ID: 2664 _ Reason for reducing amount of alcohol drunkUses data-coding 100418

Field ID: 3731 _ Former alcohol drinkerUses data-coding 100582

Field ID: 3859 _ Reason former drinker stopped drinking alcoholUses data-coding 100564

Field ID: 4462 _ Average monthly intake of other alcoholic drinksUses data-coding 100259

Field ID: 5364 _ Average weekly intake of other alcoholic drinksUses data-coding 100511

Field ID: 10818 _ Reason for reducing amount of alcohol drunk (pilot)Uses data-coding 100674

Field ID: 10853 _ Reason former drinker stopped drinking alcohol (pilot)Uses data-coding 100668

Field ID: 20117 _ Alcohol drinker statusUses data-coding 90

Field ID: 20403 _ Amount of alcohol drunk on a typical drinking dayUses data-coding 522

Field ID: 20404 _ E

['1558: Alcohol intake frequency.Uses data-coding 100402',
 '1568: Average weekly red wine intakeUses data-coding 100402',
 '1578: Average weekly champagne plus white wine intakeUses data-coding 100402',
 '1588: Average weekly beer plus cider intakeUses data-coding 100402',
 '1598: Average weekly spirits intakeUses data-coding 100402',
 '1608: Average weekly fortified wine intakeUses data-coding 100402',
 '1618: Alcohol usually taken with mealsUses data-coding 100416',
 '1628: Alcohol intake versus 10 years previouslyUses data-coding 100417',
 '2664: Reason for reducing amount of alcohol drunkUses data-coding 100418',
 '3731: Former alcohol drinkerUses data-coding 100582',
 '3859: Reason former drinker stopped drinking alcoholUses data-coding 100564',
 '4407: Average monthly red wine intakeUses data-coding 100259',
 '4418: Average monthly champagne plus white wine intakeUses data-coding 100259',
 '4429: Average monthly beer plus cider intakeUses data-coding 100259',
 '4440: Average mon

In [49]:
FID='20117'
data_coding='90'
FID_tidy_name="Alcohol Status"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])

df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Alcohol the following where selected: 
 	 {'20117'}
	 FID 20117:  Alcohol drinker statusUses data-coding 90
     20117-0.0 20117-1.0 20117-2.0
0            2      <NA>      <NA>
1            2         2         2
2            2      <NA>         2
3            2      <NA>      <NA>
4            2      <NA>      <NA>
...        ...       ...       ...
9995         2      <NA>      <NA>
9996         2      <NA>      <NA>
9997         2      <NA>      <NA>
9998         2      <NA>      <NA>
9999         2      <NA>      <NA>

[10000 rows x 3 columns]
the percentage of na
20117-0.0    0.0021
dtype: float64
Alcohol Status_Current                 9144
Alcohol Status_Never                    467
Alcohol Status_Prefer not to answer      18
Alcohol Status_Previous                 350
dtype: int64


In [50]:
word='Smoking status'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['20116'])
categories['Smoking']

Field ID: 20116 _ Smoking statusUses data-coding 90

None
['Smoking status', 'Smoking']


['1239: Current tobacco smokingUses data-coding 100347',
 '1249: Past tobacco smokingUses data-coding 100348',
 '1259: Smoking/smokers in householdUses data-coding 100370',
 '1269: Exposure to tobacco smoke at homeUses data-coding 100370',
 '1279: Exposure to tobacco smoke outside homeUses data-coding 100370',
 '2644: Light smokers, at least 100 smokes in lifetimeUses data-coding 100603',
 '2867: Age started smoking in former smokersUses data-coding 100599',
 '2877: Type of tobacco previously smokedUses data-coding 100351',
 '2887: Number of cigarettes previously smoked dailyUses data-coding 100353',
 '2897: Age stopped smokingUses data-coding 100353',
 '2907: Ever stopped smoking for 6+ monthsUses data-coding 100353',
 '2926: Number of unsuccessful stop-smoking attemptsUses data-coding 100353',
 '2936: Likelihood of resuming smokingUses data-coding 100369',
 '3436: Age started smoking in current smokersUses data-coding 100267',
 '3446: Type of tobacco currently smokedUses data-coding 

In [51]:
FID='20116'
data_coding='90'
FID_tidy_name="Smoking Status"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])

df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Smoking the following where selected: 
 	 {'20116'}
	 FID 20116:  Smoking statusUses data-coding 90
     20116-0.0 20116-1.0 20116-2.0
0            1      <NA>      <NA>
1            1         1         1
2            0      <NA>         0
3            0      <NA>      <NA>
4            1      <NA>      <NA>
...        ...       ...       ...
9995         1      <NA>      <NA>
9996         1      <NA>      <NA>
9997         1      <NA>      <NA>
9998         2      <NA>      <NA>
9999         0      <NA>      <NA>

[10000 rows x 3 columns]
the percentage of na
20116-0.0    0.0021
dtype: float64
Smoking Status_Current                 1018
Smoking Status_Never                   5467
Smoking Status_Prefer not to answer      39
Smoking Status_Previous                3455
dtype: int64


In [52]:
newdf

,50-0.0,21001-0.0,SBP_avg,DBP_avg,PR_avg,Alcohol Status_Current,Alcohol Status_Never,Alcohol Status_Prefer not to answer,Alcohol Status_Previous,Smoking Status_Current,Smoking Status_Never,Smoking Status_Prefer not to answer,Smoking Status_Previous
0,154,29.9798,136,75,82,1,0,0,0,0,0,0,1
1,174,24.4088,124,78,66,1,0,0,0,0,0,0,1
2,166,29.3584,119,76,58,1,0,0,0,0,1,0,0
3,179,30.617,170,107,56,1,0,0,0,0,1,0,0
4,168,31.3209,138,82,69,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,161,21.8356,114,64,66,1,0,0,0,0,0,0,1
9996,176,33.6067,152,78,76,1,0,0,0,0,0,0,1
9997,181,28.6011,148,88,94,1,0,0,0,0,0,0,1
9998,181,29.6999,159,92,86,1,0,0,0,1,0,0,0


In [53]:
newdf.to_csv(r'tidy_data_tiles/Anthropometrics and cardiac risk factors.csv')

In [54]:
selected_categories_tidy_columns

{'gender': {'col_name': ['31-0.0'], 'type': 'binary'},
 'birth_date': {'col_name': ['birth_date'], 'type': 'date'},
 'Townsend deprivation index at recruitment': {'col_name': ['189-0.0'],
  'type': 'float'},
 'Age at recruitment': {'col_name': ['21022-0.0'], 'type': 'int'},
 'Ethnicity': {'col_name': ['Ethnicity_African',
   'Ethnicity_Any other Asian background',
   'Ethnicity_Any other Black background',
   'Ethnicity_Any other mixed background',
   'Ethnicity_Any other white background',
   'Ethnicity_Asian or Asian British',
   'Ethnicity_Bangladeshi',
   'Ethnicity_Black or Black British',
   'Ethnicity_British',
   'Ethnicity_Caribbean',
   'Ethnicity_Chinese',
   'Ethnicity_Do not know',
   'Ethnicity_Indian',
   'Ethnicity_Irish',
   'Ethnicity_Mixed',
   'Ethnicity_Other ethnic group',
   'Ethnicity_Pakistani',
   'Ethnicity_Prefer not to answer',
   'Ethnicity_White',
   'Ethnicity_White and Asian',
   'Ethnicity_White and Black African',
   'Ethnicity_White and Black Caribbe

# 2.4 Baseline medical conditions

## 2.4.1 Read data-coding 6 from the html file to process 20002 (self-reported non-cancer illnesses)

In [55]:
newdf=pd.DataFrame()

In [56]:
word='Non-cancer illness code'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['20002'])
categories['Medical conditions']

Field ID: 20002 _ Non-cancer illness code, self-reportedUses data-coding 6

None
['Non-cancer illness code, self-reported', 'Medical conditions']


['84: Cancer year/age first occurredUses data-coding 37',
 '87: Non-cancer illness year/age first occurredUses data-coding 37',
 '134: Number of self-reported cancers',
 '135: Number of self-reported non-cancer illnesses',
 '2443: Diabetes diagnosed by doctorUses data-coding 100603',
 '2453: Cancer diagnosed by doctorUses data-coding 100603',
 '2463: Fractured/broken bones in last 5 yearsUses data-coding 100603',
 '2473: Other serious medical condition/disability diagnosed by doctorUses data-coding 100603',
 '2966: Age high blood pressure diagnosedUses data-coding 100369',
 '2976: Age diabetes diagnosedUses data-coding 100369',
 '2986: Started insulin within one year diagnosis of diabetesUses data-coding 100369',
 '3005: Fracture resulting from simple fallUses data-coding 100369',
 '3140: PregnantUses data-coding 100267',
 '3627: Age angina diagnosedUses data-coding 100563',
 '3761: Age hay fever, rhinitis or eczema diagnosedUses data-coding 100564',
 '3786: Age asthma diagnosedUses da

## Since -99999 and -1 entries cause problems this does not use the function tidy_coded_fieldID()

In [57]:
FID='20002'  #20009 is the corresponding age
data_coding='6'
FID_tidy_name="Baseline Medical conditions"

selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

From the category Medical conditions the following where selected: 
 	 {'20002'}
	 FID 20002:  Non-cancer illness code, self-reportedUses data-coding 6
     20002-0.0 20002-0.1 20002-0.2 20002-0.3 20002-0.4 20002-0.5 20002-0.6  \
0         <NA>      <NA>      <NA>      <NA>      <NA>      <NA>      <NA>   
1         <NA>      <NA>      <NA>      <NA>      <NA>      <NA>      <NA>   
2         1065      1538      <NA>      <NA>      <NA>      <NA>      <NA>   
3         1387      1628      <NA>      <NA>      <NA>      <NA>      <NA>   
4         1065      1473      1466     99999      1465      1093      1257   
...        ...       ...       ...       ...       ...       ...       ...   
9995      1387      <NA>      <NA>      <NA>      <NA>      <NA>      <NA>   
9996      <NA>      <NA>      <NA>      <NA>      <NA>      <NA>      <NA>   
9997      <NA>      <NA>      <NA>      <NA>      <NA>      <NA>      <NA>   
9998      <NA>      <NA>      <NA>      <NA>      <NA>      <NA>    

In [58]:
html_file = open(PATH_dict)
html_content = html_file.read()

bs = BeautifulSoup(html_content)
table = bs.find(lambda tag: tag.name=='table' and tag.has_attr('summary') and tag['summary']=="Coding 6") 

pd_coding=pd.read_html(str(table))[0].astype(str)

In [59]:
pd_coding

,#,Code,Meaning,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,1,-1,cardiovascular,No,1071,Top
1,2,-1,respiratory/ent,No,1072,Top
2,3,-1,gastrointestinal/abdominal,No,1073,Top
3,4,-1,renal/urology,No,1074,Top
4,5,-1,endocrine/diabetes,No,1075,Top
...,...,...,...,...,...,...
469,470,1680,bowen's disease,Yes,1776,1510
470,471,1681,hydatiform mole,Yes,1777,1391
471,472,1682,benign insulinoma,Yes,1778,1075
472,473,1683,benign neuroma,Yes,1779,1489


In [60]:
newdf=df_temp
#Read all diseases (all instances) and compare times with recruitment time. if it's before age at recruitment consider it as baseline disease otherwise it's and endline disease
disease_code_to_disease_dict=pd_coding[['Code','Meaning']].set_index('Code').to_dict()['Meaning']
#disease_code_to_disease_dict['-1'] = disease_code_to_disease_dict.pop('-1(not selectable)')
#disease_code_to_disease_dict['99999'] = disease_code_to_disease_dict.pop('99999(not selectable)')
disease_code_to_disease_dict['-1']='deleteASAP'
disease_code_to_disease_dict['99999']='deleteASAP'
#disease_code_to_disease_dict_time=[value+'_t' for value in disease_code_to_disease_dict.values()]
col_names=list(disease_code_to_disease_dict.values())#+disease_code_to_disease_dict_time
col_names.sort()
df_tidy=pd.DataFrame(index=newdf.index,columns=col_names)


In [62]:
disease_code_to_disease_dict

{'-1': 'deleteASAP',
 '1065': 'hypertension',
 '1066': 'heart/cardiac problem',
 '1067': 'peripheral vascular disease',
 '1068': 'venous thromboembolic disease',
 '1072': 'essential hypertension',
 '1073': 'gestational hypertension/pre-eclampsia',
 '1074': 'angina',
 '1075': 'heart attack/myocardial infarction',
 '1076': 'heart failure/pulmonary odema',
 '1077': 'heart arrhythmia',
 '1078': 'heart valve problem/heart murmur',
 '1079': 'cardiomyopathy',
 '1080': 'pericardial problem',
 '1081': 'stroke',
 '1082': 'transient ischaemic attack (tia)',
 '1083': 'subdural haemorrhage/haematoma',
 '1086': 'subarachnoid haemorrhage',
 '1087': 'leg claudication/ intermittent claudication',
 '1088': 'arterial embolism',
 '1093': 'pulmonary embolism +/- dvt',
 '1094': 'deep venous thrombosis (dvt)',
 '1111': 'asthma',
 '1112': 'chronic obstructive airways disease/copd',
 '1113': 'emphysema/chronic bronchitis',
 '1114': 'bronchiectasis',
 '1115': 'interstitial lung disease',
 '1117': 'other respira

In [63]:
FID='20002'
#grab columns corresponding to the first assessment x-0.y
for patientID in newdf.index:
    cols_FID=[col for col in newdf.columns if (FID in col) and col.split('-')[1].split('.')[0]=='0']
    for col in cols_FID:
        entry=newdf.loc[patientID,col]
        if not pd.isna(entry):
            df_tidy.loc[patientID,disease_code_to_disease_dict[entry]]=1
df_tidy.fillna(0,inplace=True)
df_tidy.drop(columns=['deleteASAP'],inplace=True)

In [64]:
df_tidy.sum()

abdominal hernia                                         8
abnormal smear (cervix)                                  7
acne/acne vulgaris                                       6
acromegaly                                               0
acute infective polyneuritis/guillain-barre syndrome     6
                                                        ..
vocal cord polyp                                         3
wegners granulmatosis                                    3
whooping cough / pertussis                              12
wolff parkinson white / wpw syndrome                     1
yellow fever                                             0
Length: 445, dtype: int64

In [65]:
word='Rheumatic aortic '
for col in df_tidy.columns:
    if word in col.lower():
        print(col)

### Exclusions

disease exclusion: 
1. 'atrial fibrillation'
2. 'atrial flutter'
3. 'aortic valve disease' 
4. 'aortic regurgitation / incompetence',
5. 'aortic stenosis' 
*6. 'heart attack/myocardial infarction'
*7. 'heart failure/pulmonary odema'
*8. 'heart valve problem/heart murmur'
9. 'heart arrhythmia'
10. 'irregular heart beat'
11. 'hypertrophic cardiomyopathy (hcm / hocm)'
*12. 'heart/cardiac problem'
13. 'cardiomyopathy'
14. 'myocarditis'
*15. 'ischaemic stroke'
16. 'mitral regurgitation / incompetence'
17. 'mitral stenosis'
18. 'mitral valve disease'
19. 'mitral valve prolapse'
20. 'svt / supraventricular tachycardia'


How about:
1. 'aortic aneurysm',
2. 'aortic aneurysm rupture'
3. 'aortic dissection'
7. 'arterial embolism'  
*9. 'peripheral vascular disease'
11. 'varicose ulcer'
12. 'varicose veins'
13. 'venous thromboembolic disease'
14. deep venous thrombosis (dvt)
15. pericardial effusion
16. pericardial problem
17. pericarditis
microscopic polyarteritis
polyartertis nodosa

predictor:
1. 'hypertension'


"""

In [57]:
exclude_self_reported=['atrial fibrillation',
'atrial flutter',
'aortic valve disease',
'aortic regurgitation / incompetence',
'aortic stenosis',
'heart attack/myocardial infarction',
'heart failure/pulmonary odema',
'heart valve problem/heart murmur',
'heart arrhythmia',
'irregular heart beat',
'hypertrophic cardiomyopathy (hcm / hocm)',
'heart/cardiac problem',
'cardiomyopathy',
'myocarditis',
'ischaemic stroke',
'mitral regurgitation / incompetence',
'mitral stenosis',
'mitral valve disease',
'mitral valve prolapse',
'svt / supraventricular tachycardia']

In [58]:
df_tidy['Exclude_baseline_MedCon']=df_tidy[exclude_self_reported].sum(axis=1)
df_tidy.drop(columns=exclude_self_reported,inplace=True)

In [59]:
newdf=df_tidy

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}
selected_categories_exclusions.append('Exclude_baseline_MedCon')

In [60]:
newdf.to_csv(r'tidy_data_tiles/Baseline Medical Conditions.csv')

# 2.5  Home Location

In [61]:
word='home'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['20074'])
categories['Home locations']

Field ID: 777 _ Frequency of travelling from home to job workplaceUses data-coding 100298

Field ID: 796 _ Distance between home and job workplaceUses data-coding 100300

Field ID: 1269 _ Exposure to tobacco smoke at homeUses data-coding 100370

Field ID: 1279 _ Exposure to tobacco smoke outside homeUses data-coding 100370

Field ID: 6140 _ Heating type(s) in homeUses data-coding 100289

Field ID: 20074 _ Home location at assessment - east co-ordinate (rounded)

Field ID: 20075 _ Home location at assessment - north co-ordinate (rounded)

Field ID: 20109 _ Ingredients in homemade soupUses data-coding 86

Field ID: 20118 _ Home area population density - urban or ruralUses data-coding 91

Field ID: 22702 _ Home location - east co-ordinate (rounded)

Field ID: 22704 _ Home location - north co-ordinate (rounded)

Field ID: 102620 _ Homemade soup intakeUses data-coding 100005

None
['Home location at assessment - east co-ordinate (rounded)', 'Reception']


['22700: Date first recorded at locationUses data-coding 1313',
 '22702: Home location - east co-ordinate (rounded)',
 '22704: Home location - north co-ordinate (rounded)']

In [62]:
FID='22700'  #53 is the corresponding age
FID_tidy_name="Home location EC"

selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config).apply(pd.to_datetime)

for index in df_temp.index:
    df_temp.iloc[index]
    

#Get date first attended
selected_categories1={fieldID_to_ctgry_dict['53'][1]:['include','53']}
process_config1={'53':{'type':'string'}}

df_date_assessment=pd.to_datetime(preprocess(selected_categories1,categories,
                                             UID_to_dscrpt,df,process_config1)['53-0.0'],format='%Y-%m-%d').to_frame()

print(df_temp)
print(df_date_assessment)
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

From the category Home locations the following where selected: 
 	 {'22700'}
	 FID 22700:  Date first recorded at locationUses data-coding 1313
From the category Reception the following where selected: 
 	 {'53'}
	 FID 53:  Date of attending assessment centre
        22700-0.0  22700-0.1 22700-0.2 22700-0.3 22700-0.4 22700-0.5  \
0      1994-12-07        NaT       NaT       NaT       NaT       NaT   
1      2002-07-01        NaT       NaT       NaT       NaT       NaT   
2      1994-05-10 2014-04-11       NaT       NaT       NaT       NaT   
3      1997-04-04        NaT       NaT       NaT       NaT       NaT   
4      1991-01-01        NaT       NaT       NaT       NaT       NaT   
...           ...        ...       ...       ...       ...       ...   
502522 1996-07-03        NaT       NaT       NaT       NaT       NaT   
502523 1987-10-08        NaT       NaT       NaT       NaT       NaT   
502524 2002-06-05        NaT       NaT       NaT       NaT       NaT   
502525 1984-02-10   

In [63]:
df_valid=pd.DataFrame(index=df_temp.index,columns=['Exclude_Home_location','Home_Longitude','Home_Latitude'])
for index in df_temp.index:
    df_valid.loc[index,'Exclude_Home_location']=any((df_temp.loc[index,df_temp.iloc[index].notna()] > df_date_assessment.loc[index,'53-0.0']) & (df_temp.loc[index,df_temp.iloc[index].notna()]< df_date_assessment.loc[index,'53-0.0']+pd.Timedelta(weeks=261)))#checks if the last date for which I have a registered address is smaller than the date of assessment. If this is not true it means after the assessment the address has changes and the patient should be excluded
df_valid['Home_Longitude']=df['20074-0.0']
df_valid['Home_Latitude']=df['20075-0.0']

In [64]:
df_valid

,Exclude_Home_location,Home_Longitude,Home_Latitude
0,False,340000,388000
1,False,337000,388000
2,True,524000,168000
3,False,512000,172000
4,False,379000,417000
...,...,...,...
502522,False,372000,394000
502523,False,519000,166000
502524,False,345000,390000
502525,False,385000,410000


In [65]:
selected_categories_tidy['Home locations']=['Home location at assessment']
selected_categories_tidy_columns['Home location at assessment']={"col_name":['Home_Longitude','Home_Latitude'],"type":"int"}
selected_categories_exclusions.append("Exclude_Home_location")

## 2.5.1 Add 20118 _ Home area population density

In [66]:
FID='20118'  #53 is the corresponding age
FID_tidy_name="Home area population density"

selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

From the category Reception the following where selected: 
 	 {'20118'}
	 FID 20118:  Home area population density - urban or ruralUses data-coding 91


In [67]:
df_temp

,20118-0.0
0,5
1,5
2,5
3,5
4,5
...,...
502522,5
502523,5
502524,5
502525,5


In [68]:
newdf=pd.concat([df_valid,df_temp],axis=1)

selected_categories_tidy['Home locations'].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_temp.columns.values),"type":"int"}

In [69]:
newdf

,Exclude_Home_location,Home_Longitude,Home_Latitude,20118-0.0
0,False,340000,388000,5
1,False,337000,388000,5
2,True,524000,168000,5
3,False,512000,172000,5
4,False,379000,417000,5
...,...,...,...,...
502522,False,372000,394000,5
502523,False,519000,166000,5
502524,False,345000,390000,5
502525,False,385000,410000,5


In [70]:
newdf.to_csv(r'tidy_data_tiles/Home locations.csv')

In [71]:
selected_categories_tidy_columns

{'gender': {'col_name': ['31-0.0'], 'type': 'binary'},
 'birth_date': {'col_name': ['birth_date'], 'type': 'date'},
 'Townsend deprivation index at recruitment': {'col_name': ['189-0.0'],
  'type': 'float'},
 'Age at recruitment': {'col_name': ['21022-0.0'], 'type': 'int'},
 'Ethnicity': {'col_name': ['Ethnicity_African',
   'Ethnicity_Any other Asian background',
   'Ethnicity_Any other Black background',
   'Ethnicity_Any other mixed background',
   'Ethnicity_Any other white background',
   'Ethnicity_Asian or Asian British',
   'Ethnicity_Bangladeshi',
   'Ethnicity_Black or Black British',
   'Ethnicity_British',
   'Ethnicity_Caribbean',
   'Ethnicity_Chinese',
   'Ethnicity_Do not know',
   'Ethnicity_Indian',
   'Ethnicity_Irish',
   'Ethnicity_Mixed',
   'Ethnicity_Other ethnic group',
   'Ethnicity_Pakistani',
   'Ethnicity_Prefer not to answer',
   'Ethnicity_White',
   'Ethnicity_White and Asian',
   'Ethnicity_White and Black African',
   'Ethnicity_White and Black Caribbe

# 2.6 Pollutants

In [72]:
word='poll'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['24003'])
categories['Residential air pollution']  

Field ID: 24003 _ Nitrogen dioxide air pollution; 2010

Field ID: 24004 _ Nitrogen oxides air pollution; 2010

Field ID: 24005 _ Particulate matter air pollution (pm10); 2010

Field ID: 24006 _ Particulate matter air pollution (pm2.5); 2010

Field ID: 24007 _ Particulate matter air pollution (pm2.5) absorbance; 2010

Field ID: 24008 _ Particulate matter air pollution 2.5-10um; 2010

Field ID: 24016 _ Nitrogen dioxide air pollution; 2005

Field ID: 24017 _ Nitrogen dioxide air pollution; 2006

Field ID: 24018 _ Nitrogen dioxide air pollution; 2007

Field ID: 24019 _ Particulate matter air pollution (pm10); 2007

Field ID: 24020 _ Average daytime sound level of noise pollution

Field ID: 24021 _ Average evening sound level of noise pollution

Field ID: 24022 _ Average night-time sound level of noise pollution

None
['Nitrogen dioxide air pollution; 2010', 'Residential air pollution']


['24003: Nitrogen dioxide air pollution; 2010',
 '24004: Nitrogen oxides air pollution; 2010',
 '24005: Particulate matter air pollution (pm10); 2010',
 '24006: Particulate matter air pollution (pm2.5); 2010',
 '24007: Particulate matter air pollution (pm2.5) absorbance; 2010',
 '24008: Particulate matter air pollution 2.5-10um; 2010',
 '24009: Traffic intensity on the nearest road',
 '24010: Inverse distance to the nearest road',
 '24011: Traffic intensity on the nearest major road',
 '24012: Inverse distance to the nearest major road',
 '24013: Total traffic load on major roads',
 '24014: Close to major roadUses data-coding 12',
 '24015: Sum of road length of major roads within 100m',
 '24016: Nitrogen dioxide air pollution; 2005',
 '24017: Nitrogen dioxide air pollution; 2006',
 '24018: Nitrogen dioxide air pollution; 2007',
 '24019: Particulate matter air pollution (pm10); 2007']

## 2.6.1 Pollutants

In [73]:
#PM 24005  24006 24007 24008   NOx 24003  24004  Noise   24020 24021 24022

In [74]:
selected_categories={'Residential air pollution':['include','24003', '24004','24005',  '24006', '24007', '24008']}
process_config={'24005':{'type':'string'},'24006':{'type':'string'},'24007':{'type':'string'},'24008':{'type':'string'},
                '24003':{'type':'string'},'24004':{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)

newdf=df_temp

selected_categories_tidy_temp={'Residential air pollution':['24003: Nitrogen dioxide air pollution; 2010',
 '24004: Nitrogen oxides air pollution; 2010',
 '24005: Particulate matter air pollution (pm10); 2010',
 '24006: Particulate matter air pollution (pm2.5); 2010',
 '24007: Particulate matter air pollution (pm2.5) absorbance; 2010',
 '24008: Particulate matter air pollution 2.5-10um; 2010']}
selected_categories_tidy_columns_temp={'24003: Nitrogen dioxide air pollution; 2010':{"col_name":['24003-0.0'],"type":'float'},
                                       '24004: Nitrogen oxides air pollution; 2010':{"col_name":['24004-0.0'],"type":'float'},
                                       '24005: Particulate matter air pollution (pm10); 2010':{"col_name":['24005-0.0'],"type":'float'},
                                       '24006: Particulate matter air pollution (pm2.5); 2010':{"col_name":['24006-0.0'],"type":'float'},
                                       '24007: Particulate matter air pollution (pm2.5) absorbance; 2010':{"col_name":['24007-0.0'],"type":'float'},
                                       '24008: Particulate matter air pollution 2.5-10um; 2010':{"col_name":['24008-0.0'],"type":'float'}}


From the category Residential air pollution the following where selected: 
 	 {'24007', '24003', '24005', '24006', '24008', '24004'}
	 FID 24007:  Particulate matter air pollution (pm2.5) absorbance; 2010
	 FID 24003:  Nitrogen dioxide air pollution; 2010
	 FID 24005:  Particulate matter air pollution (pm10); 2010
	 FID 24006:  Particulate matter air pollution (pm2.5); 2010
	 FID 24008:  Particulate matter air pollution 2.5-10um; 2010
	 FID 24004:  Nitrogen oxides air pollution; 2010
       24007-0.0 24003-0.0 24005-0.0          24006-0.0 24008-0.0 24004-0.0
0           1.23     31.57     14.48               10.4      5.68     53.83
1           1.26     29.36     15.93  9.960000000000001      6.02     45.93
2            1.3     29.68     16.84               9.27      6.42     46.93
3           1.58      33.7     19.17               11.6      7.79     60.85
4           0.97     25.46      14.9              10.85      5.87     45.61
...          ...       ...       ...                ...

## 2.6.2 Road 
24009: Traffic intensity on the nearest road',
 '24010: Inverse distance to the nearest road',
 '24011: Traffic intensity on the nearest major road',
 '24012: Inverse distance to the nearest major road',
 '24013: Total traffic load on major roads',
 '24014: Close to major roadUses data-coding 7 comprises 2 Integer-valued members in a simple list.',
 '24015: Sum of road length of major roads within 100m',

In [75]:
selected_categories={'Residential air pollution':['include','24009',  '24010', '24011', '24012',
                                                 '24013', '24014','24015']}
process_config={'24009':{'type':'string'},'24010':{'type':'string'},'24011':{'type':'string'},'24012':{'type':'string'},
                '24013':{'type':'string'},'24014':{'type':'string'},'24015':{'type':'string'}}
df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

From the category Residential air pollution the following where selected: 
 	 {'24010', '24011', '24015', '24014', '24012', '24009', '24013'}
	 FID 24010:  Inverse distance to the nearest road
	 FID 24011:  Traffic intensity on the nearest major road
	 FID 24015:  Sum of road length of major roads within 100m
	 FID 24014:  Close to major roadUses data-coding 12
	 FID 24012:  Inverse distance to the nearest major road
	 FID 24009:  Traffic intensity on the nearest road
	 FID 24013:  Total traffic load on major roads


In [76]:
newdf=pd.concat([newdf,df_temp],axis=1)

In [77]:
#newdf['24014-0.0'].fillna('0',inplace=True)

In [78]:
selected_categories_tidy_temp['Residential air pollution']+=[ '24009: Traffic intensity on the nearest road',
 '24010: Inverse distance to the nearest road',
 '24011: Traffic intensity on the nearest major road',
 '24012: Inverse distance to the nearest major road',
 '24013: Total traffic load on major roads',
 '24014: Close to major roadUses data-coding 7 comprises 2 Integer-valued members in a simple list.',
 '24015: Sum of road length of major roads within 100m']
selected_categories_tidy_columns_temp1={'24009: Traffic intensity on the nearest road':{"col_name":['24009-0.0'],"type":'int'},
                                       '24010: Inverse distance to the nearest road':{"col_name":['24010-0.0'],"type":'float'},
                                       '24011: Traffic intensity on the nearest major road':{"col_name":['24011-0.0'],"type":'int'},
                                       '24012: Inverse distance to the nearest major road':{"col_name":['24012-0.0'],"type":'float'},
                                       '24013: Total traffic load on major roads':{"col_name":['24013-0.0'],"type":'int'},
                                       '24014: Close to major road':{"col_name":['24014-0.0'],"type":'binary'},
                                       '24015: Sum of road length of major roads within 100m':{"col_name":['24015-0.0'],"type":'float'}}

selected_categories_tidy_columns_temp={**selected_categories_tidy_columns_temp,**selected_categories_tidy_columns_temp1}

# 2.6.2.2  Annual average road traffic noise level for year 2009 !

In [79]:
word='noise'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['24022'])
categories['Residential noise pollution']  

Field ID: 24020 _ Average daytime sound level of noise pollution

Field ID: 24021 _ Average evening sound level of noise pollution

Field ID: 24022 _ Average night-time sound level of noise pollution

None
['Average night-time sound level of noise pollution', 'Residential noise pollution']


['24020: Average daytime sound level of noise pollution',
 '24021: Average evening sound level of noise pollution',
 '24022: Average night-time sound level of noise pollution']

In [80]:
selected_categories={'Residential noise pollution':['include','24020',  '24021', '24022']}
process_config={'24020':{'type':'string'},'24021':{'type':'string'},'24022':{'type':'string'}}
df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

From the category Residential noise pollution the following where selected: 
 	 {'24022', '24021', '24020'}
	 FID 24022:  Average night-time sound level of noise pollution
	 FID 24021:  Average evening sound level of noise pollution
	 FID 24020:  Average daytime sound level of noise pollution


In [81]:
newdf=pd.concat([newdf,df_temp],axis=1)

In [82]:
selected_categories_tidy_temp['Residential noise pollution']=[ '24020: Average daytime sound level of noise pollution',
                                                             '24021: Average evening sound level of noise pollution',
                                                             '24022: Average night-time sound level of noise pollution']
selected_categories_tidy_columns_temp1={'24020: Average daytime sound level of noise pollution':{"col_name":['24020-0.0'],"type":'float'},
                                       '24021: Average evening sound level of noise pollution':{"col_name":['24021-0.0'],"type":'float'},
                                       '24022: Average night-time sound level of noise pollution':{"col_name":['24022-0.0'],"type":'float'}}

selected_categories_tidy_columns_temp={**selected_categories_tidy_columns_temp,**selected_categories_tidy_columns_temp1}

### 2.6.2.3 other env exposures: green space, natural environment, domestic garden, and water within 300 meters and 1000 meters of residential addresses!

In [83]:
word='green'
print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['24503'])
categories['Greenspace and coastal proximity']  

Field ID: 24500 _ Greenspace percentage, buffer 1000m

Field ID: 24503 _ Greenspace percentage, buffer 300m

Field ID: 100420 _ Green tea intakeUses data-coding 100007

Field ID: 104120 _ Green bean intakeUses data-coding 100013

None
['Greenspace percentage, buffer 300m', 'Greenspace and coastal proximity']


['24500: Greenspace percentage, buffer 1000m',
 '24501: Domestic garden percentage, buffer 1000m',
 '24502: Water percentage, buffer 1000m',
 '24503: Greenspace percentage, buffer 300m',
 '24504: Domestic garden percentage, buffer 300m',
 '24505: Water percentage, buffer 300m',
 '24506: Natural environment percentage, buffer 1000m',
 '24507: Natural environment percentage, buffer 300m',
 '24508: Distance (Euclidean) to coast']

In [84]:
selected_categories={'Greenspace and coastal proximity':['include','24500',"24501","24502","24503","24504","24505",
                                                        "24506","24507","24508"]}
process_config={'24500':{'type':'string'},'24501':{'type':'string'},'24502':{'type':'string'},
               '24503':{'type':'string'},'24504':{'type':'string'},'24505':{'type':'string'},
               '24506':{'type':'string'},'24507':{'type':'string'},'24508':{'type':'string'}}
df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

From the category Greenspace and coastal proximity the following where selected: 
 	 {'24507', '24504', '24505', '24508', '24503', '24501', '24506', '24500', '24502'}
	 FID 24507:  Natural environment percentage, buffer 300m
	 FID 24504:  Domestic garden percentage, buffer 300m
	 FID 24505:  Water percentage, buffer 300m
	 FID 24508:  Distance (Euclidean) to coast
	 FID 24503:  Greenspace percentage, buffer 300m
	 FID 24501:  Domestic garden percentage, buffer 1000m
	 FID 24506:  Natural environment percentage, buffer 1000m
	 FID 24500:  Greenspace percentage, buffer 1000m
	 FID 24502:  Water percentage, buffer 1000m


In [85]:
df_temp=df_temp[['24500-0.0','24501-0.0','24502-0.0','24503-0.0','24504-0.0','24505-0.0','24506-0.0','24507-0.0','24508-0.0']]


In [86]:
df_temp.head()

,24500-0.0,24501-0.0,24502-0.0,24503-0.0,24504-0.0,24505-0.0,24506-0.0,24507-0.0,24508-0.0
0,27.692,32.059,0.13,52.041,15.619,0.102,15.784,6.25,10.712
1,30.191,23.259,0.67,29.911,31.943,0.127,22.432,17.411,8.414999999999999
2,41.897,28.888,0.424,8.345000000000001,50.888,0.043,38.714,1.562,42.882
3,31.164,26.188,1.158,10.417,39.355,0.504,20.86,3.348,54.123
4,63.647,13.553,1.059,50.915,24.277,0.294,59.275,45.089,40.685


In [87]:
newdf=pd.concat([newdf,df_temp],axis=1)

In [88]:
selected_categories_tidy_temp['Greenspace and coastal proximity']=[ '24504:  Domestic garden percentage, buffer 300m',
                                                             '24502:  Water percentage, buffer 1000m',
                                                             '24501:  Domestic garden percentage, buffer 1000m',
                                                                  '24505:  Water percentage, buffer 300m',
                                                                  '24507:  Natural environment percentage, buffer 300m',
                                                                  '24508:  Distance (Euclidean) to coast',
                                                                  '24503:  Greenspace percentage, buffer 300m',
                                                                  '24506:  Natural environment percentage, buffer 1000m',
                                                                  '24500:  Greenspace percentage, buffer 1000m']
selected_categories_tidy_columns_temp1={'24504:  Domestic garden percentage, buffer 300m':{"col_name":['24504-0.0'],"type":'float'},
                                       '24502:  Water percentage, buffer 1000m':{"col_name":['24502-0.0'],"type":'float'},
                                       '24501:  Domestic garden percentage, buffer 1000m':{"col_name":['24501-0.0'],"type":'float'},
                                       '24505:  Water percentage, buffer 300m':{"col_name":['24505-0.0'],"type":'float'},
                                       '24507:  Natural environment percentage, buffer 300m':{"col_name":['24507-0.0'],"type":'float'},
                                       '24508:  Distance (Euclidean) to coast':{"col_name":['24508-0.0'],"type":'float'},
                                       '24503:  Greenspace percentage, buffer 300m':{"col_name":['24503-0.0'],"type":'float'},
                                       '24506:  Natural environment percentage, buffer 1000m':{"col_name":['24506-0.0'],"type":'float'},
                                       '24500:  Greenspace percentage, buffer 1000m':{"col_name":['24500-0.0'],"type":'float'}}

selected_categories_tidy_columns_temp={**selected_categories_tidy_columns_temp,**selected_categories_tidy_columns_temp1}

In [89]:
newdf

,24007-0.0,24003-0.0,24005-0.0,24006-0.0,24008-0.0,24004-0.0,24010-0.0,24011-0.0,24015-0.0,24014-0.0,...,24020-0.0,24500-0.0,24501-0.0,24502-0.0,24503-0.0,24504-0.0,24505-0.0,24506-0.0,24507-0.0,24508-0.0
0,1.23,31.57,14.48,10.4,5.68,53.83,0.04,15763,0,0,...,56.99,27.692,32.059,0.13,52.041,15.619,0.102,15.784,6.25,10.712
1,1.26,29.36,15.93,9.960000000000001,6.02,45.93,0.04,14247,0,0,...,54.65,30.191,23.259,0.67,29.911,31.943,0.127,22.432,17.411,8.414999999999999
2,1.3,29.68,16.84,9.27,6.42,46.93,0.03,26109,0,0,...,52.85,41.897,28.888,0.424,8.345000000000001,50.888,0.043,38.714,1.562,42.882
3,1.58,33.7,19.17,11.6,7.79,60.85,0.06,67683,0,0,...,57.87,31.164,26.188,1.158,10.417,39.355,0.504,20.86,3.348,54.123
4,0.97,25.46,14.9,10.85,5.87,45.61,0.04,15765,0,0,...,53.63,63.647,13.553,1.059,50.915,24.277,0.294,59.275,45.089,40.685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502522,0.93,20.62,20.34,9.76,9.08,33.79,0.04,120415,0,0,...,53.27,60.402,16.644,1.321,60.348,20.773,0.089,49.403,37.946,39.455
502523,1.78,34.07,17.51,10.96,6.7,55.22,0.03,28597,0,0,...,54.94,16.926,43.468,0.281,28.521,31.619,0.467,11.943,23.214,47.666
502524,1.37,31.22,15.22,10.86,5.78,48.16,0.06,12985,0,0,...,58.04,50.731,17.855,0.763,35.892,35.009,0.502,38.602,15.625,14.004
502525,1.04,21.87,14.83,9.029999999999999,5.92,34.8,0.06,17061,0,0,...,54.53,44.897,20.899,0.075,43.218,27.409,0.051,35.131,59.821,48.817


In [90]:
selected_categories_tidy={**selected_categories_tidy,**selected_categories_tidy_temp}
selected_categories_tidy_columns={**selected_categories_tidy_columns,**selected_categories_tidy_columns_temp}

In [91]:
newdf.to_csv(r'tidy_data_tiles/PollutantsAndEnv.csv')

In [92]:
selected_categories_tidy

{'demographics and socioeconomics': ['gender',
  'birth_date',
  'Townsend deprivation index at recruitment',
  'Age at recruitment',
  'Ethnicity',
  'Household Income',
  'Current employment status',
  'Qualifications'],
 'Body size measures': ['Standing height', 'BMI'],
 'Blood pressure': ['SBP', 'DBP', 'PR'],
 'Alcohol': ['Alcohol Status'],
 'Smoking': ['Smoking Status'],
 'Medical conditions': ['Baseline Medical conditions'],
 'Home locations': ['Home location at assessment',
  'Home area population density'],
 'Residential air pollution': ['24003: Nitrogen dioxide air pollution; 2010',
  '24004: Nitrogen oxides air pollution; 2010',
  '24005: Particulate matter air pollution (pm10); 2010',
  '24006: Particulate matter air pollution (pm2.5); 2010',
  '24007: Particulate matter air pollution (pm2.5) absorbance; 2010',
  '24008: Particulate matter air pollution 2.5-10um; 2010',
  '24009: Traffic intensity on the nearest road',
  '24010: Inverse distance to the nearest road',
  '2401

In [93]:
def save_obj(obj, name ):
    with open('obj_'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj_' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [94]:
import pickle
save_obj(selected_categories_tidy,'selected_categories_tidy')
save_obj(selected_categories_tidy_columns,'selected_categories_tidy_columns')
save_obj(selected_categories_exclusions,'selected_categories_exclusions')

In [95]:
load_obj('selected_categories_exclusions')

['Exclude_baseline_MedCon', 'Exclude_Home_location']

In [96]:
selected_categories_tidy_columns

{'gender': {'col_name': ['31-0.0'], 'type': 'binary'},
 'birth_date': {'col_name': ['birth_date'], 'type': 'date'},
 'Townsend deprivation index at recruitment': {'col_name': ['189-0.0'],
  'type': 'float'},
 'Age at recruitment': {'col_name': ['21022-0.0'], 'type': 'int'},
 'Ethnicity': {'col_name': ['Ethnicity_African',
   'Ethnicity_Any other Asian background',
   'Ethnicity_Any other Black background',
   'Ethnicity_Any other mixed background',
   'Ethnicity_Any other white background',
   'Ethnicity_Asian or Asian British',
   'Ethnicity_Bangladeshi',
   'Ethnicity_Black or Black British',
   'Ethnicity_British',
   'Ethnicity_Caribbean',
   'Ethnicity_Chinese',
   'Ethnicity_Do not know',
   'Ethnicity_Indian',
   'Ethnicity_Irish',
   'Ethnicity_Mixed',
   'Ethnicity_Other ethnic group',
   'Ethnicity_Pakistani',
   'Ethnicity_Prefer not to answer',
   'Ethnicity_White',
   'Ethnicity_White and Asian',
   'Ethnicity_White and Black African',
   'Ethnicity_White and Black Caribbe

# Additional inclusions for Jing

In [ ]:
selected_categories={'Residential air pollution':['include','24009',  '24010', '24011', '24012',
                                                 '24013', '24014','24015']}
process_config={'24009':{'type':'string'},'24010':{'type':'string'},'24011':{'type':'string'},'24012':{'type':'string'},
                '24013':{'type':'string'},'24014':{'type':'string'},'24015':{'type':'string'}}
df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

In [66]:
#word='green'
#print(search_description(word,UID_to_dscrpt))
print(fieldID_to_ctgry_dict['6150'])
#categories['Greenspace and coastal proximity']  

['Vascular/heart problems diagnosed by doctor', 'Medical conditions']


In [ ]:
selected_categories={'Additional Data':['include','24009',  '24010', '24011', '24012',
                                                 '24013', '24014','24015']}
process_config={'24009':{'type':'string'},'24010':{'type':'string'},'24011':{'type':'string'},'24012':{'type':'string'},
                '24013':{'type':'string'},'24014':{'type':'string'},'24015':{'type':'string'}}
df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

In [67]:
df["6150-0.0"]

0       -7
1       -7
2        4
3       -7
4        4
        ..
9995    -7
9996    -7
9997    -7
9998    -7
9999    -7
Name: 6150-0.0, Length: 10000, dtype: object

In [68]:
FID='6150'  #20009 is the corresponding age
data_coding='100605'
FID_tidy_name="Vascular/heart_problems_diagnosed_by_doctor"

selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

From the category Medical conditions the following where selected: 
 	 {'6150'}
	 FID 6150:  Vascular/heart problems diagnosed by doctorUses data-coding 100605
     6150-0.0 6150-0.1 6150-0.2 6150-0.3 6150-1.0 6150-1.1 6150-1.2 6150-1.3  \
0          -7     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
1          -7     <NA>     <NA>     <NA>       -7     <NA>     <NA>     <NA>   
2           4     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
3          -7     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
4           4     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
...       ...      ...      ...      ...      ...      ...      ...      ...   
9995       -7     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9996       -7     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9997       -7     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
9998       -7     <NA>     <NA>     <NA>

In [69]:
html_file = open(PATH_dict)
html_content = html_file.read()

bs = BeautifulSoup(html_content)
table = bs.find(lambda tag: tag.name=='table' and tag.has_attr('summary') and tag['summary']=="Coding 100605") 

pd_coding=pd.read_html(str(table))[0].astype(str)

In [70]:
pd_coding

,#,Code,Meaning
0,1,1,Heart attack
1,2,2,Angina
2,3,3,Stroke
3,4,4,High blood pressure
4,5,-7,None of the above
5,6,-3,Prefer not to answer
